In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

import data_utils
import train_utils
import models

%reload_ext autoreload
%autoreload 2

In [2]:
dev = data_utils.COAD_dataset(data_utils.COAD_DEV)
dev_loader = torch.utils.data.DataLoader(dev, batch_size=1, shuffle=True, pin_memory=True)
train = data_utils.COAD_dataset(data_utils.COAD_TRAIN)
train_loader = torch.utils.data.DataLoader(train, batch_size=1, shuffle=True, pin_memory=True)
valid = data_utils.COAD_dataset(data_utils.COAD_VALID)
valid_loader = torch.utils.data.DataLoader(valid, batch_size=1, shuffle=False, pin_memory=True)

Generator(
  (m): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu): ReLU()
  (conv): Sequential(
    (0): Conv2d(3, 36, kernel_size=(4, 4), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(36, 48, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (lstm): LSTM(1200, 512, num_layers=2, batch_first=True, bidirectional=True)
  (classification_layer): Linear(in_features=1024, out_features=2, bias=True)
)

In [132]:
n_conv_layers = 2
kernel_size = [4,3]
n_conv_filters = [36,48]
hidden_size = 512
n_rnn_layers = 2
dropout=0.0
gen = models.Generator(n_conv_layers, kernel_size, n_conv_filters, hidden_size, n_rnn_layers, dropout=dropout)
gen.cuda()

n_conv_layers = 2
n_fc_layers = 2
kernel_size = [4,3]
n_conv_filters = [36,48]
hidden_size = [512,512]
dropout=0.0
enc = models.ConvNet(n_conv_layers, n_fc_layers, kernel_size, n_conv_filters, hidden_size, dropout=dropout)
enc.cuda()


num_samples = 10
lamb1 = 0.0
lamb2 = 0.0
xent = nn.CrossEntropyLoss()
learning_rate = 1e-4


params = list(enc.parameters()) + list(gen.parameters())
optimizer = torch.optim.Adam(params, lr = learning_rate)

In [7]:
def pool_fn(x):
    #v,a = torch.max(x,0)
    v = torch.mean(x,0)
    return v

In [7]:
epochs = 10
for e in range(epochs):
    train_utils.rationales_training_loop(e, train_loader, gen, enc, pool_fn, num_samples, 
                                         lamb1, lamb2, xent, learning_rate, optimizer)
    loss = train_utils.rationales_validation_loop(e, valid_loader, gen, enc, pool_fn, num_samples, 
                                                  lamb1, lamb2, xent)

Epoch: 0, Train Loss: 7594282.3017
Epoch: 0, Val Loss: 141.4314, Val Acc: 0.4082
Epoch: 1, Train Loss: 1362.6312
Epoch: 1, Val Loss: 138.9712, Val Acc: 0.4082


RuntimeError: cuDNN error: CUDNN_STATUS_EXECUTION_FAILED

In [9]:
# THIS MAY NOT WORK ON EPOCH 1!!! BECAUSE OF CUDA ERROR!!!
torch.backends.cudnn.version()
lsm = torch.nn.LogSoftmax(dim=2)

In [118]:
    gen.train()
    enc.train()
    
    total_loss = 0
    
    for slide,label in train_loader:
        slide,label = slide.squeeze(0).cuda(),label.cuda()
        break

In [11]:
def sample_gumbel(shape, eps=1e-20): 
    """Sample from Gumbel(0, 1)"""
    U = torch.rand(shape,dtype=torch.float32,device='cuda')
    return -torch.log(-torch.log(U + eps) + eps)

In [12]:
def gumbel_softmax_sample(logits, temperature): 
    """ Draw a sample from the Gumbel-Softmax distribution"""
    y = logits + sample_gumbel(logits.shape)
    return F.softmax( y / temperature,dim=1)


In [13]:
def gumbel_softmax(logits, temperature, hard=False):
    """Sample from the Gumbel-Softmax distribution and optionally discretize.
  Args:
    logits: [batch_size, n_class] unnormalized log-probs
    temperature: non-negative scalar
    hard: if True, take argmax, but differentiate w.r.t. soft sample y
  Returns:
    [batch_size, n_class] sample from the Gumbel-Softmax distribution.
    If hard=True, then the returned sample will be one-hot, otherwise it will
    be a probabilitiy distribution that sums to 1 across classes
  """
    y = gumbel_softmax_sample(logits, temperature)
    if hard:
        y = torch.argmax(logits,dim=1)
    return y

In [119]:
preds = gen(slide)

In [120]:
logits = lsm(preds).squeeze(0)

In [121]:
sample = gumbel_softmax(logits, 10.0)
#sample = torch.zeros_like(sample)+0.5

In [129]:
loss

tensor(0.6809, device='cuda:0', grad_fn=<NllLossBackward>)

In [123]:
rationale = slide.view(3,27,27,-1) * sample[:,1]
rationale = rationale.view(-1,slide.shape[1],slide.shape[2],slide.shape[3])

In [124]:
output = enc(rationale)

In [125]:
pool = pool_fn(output)

In [126]:
y_hat = enc.classification_layer(pool)

In [128]:
loss = xent(y_hat.unsqueeze(0),label)

In [130]:
loss.backward()
optimizer.step()

In [131]:
for p in params:
    print(p.grad)
optimizer.zero_grad()

tensor([[[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],

         [[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],

         [[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]]],


        [[[-1.4824e-03, -1.2373e-03, -1.7632e-03, -1.5996e-03],
          [-1.4752e-03, -1.2300e-03, -1.8194e-03, -1.4933e-03],
          [-1.3611e-03, -1.1104e-03, -1.6497e-03, -1.4167e-03],
          [-1.4463e-03, -1.1894e

In [136]:
def rationales_training_loop_GS(e, train_loader, gen, enc, pool_fn, lamb1, lamb2, xent, learning_rate, optimizer, parameters):
    gen.train()
    enc.train()
    
    total_loss = 0
    
    for slide,label in train_loader:
        slide,label = slide.squeeze(0).cuda(),label.cuda()
        
        # generate tile rationales
        preds = gen(slide)
        logits = lsm(preds).squeeze(0)
        sample = gumbel_softmax(logits, temperature=10.0)
        rationale = slide.view(slide.shape[1],slide.shape[2],slide.shape[3],-1) * sample[:,1]
        rationale = rationale.view(-1,slide.shape[1],slide.shape[2],slide.shape[3])
        
        # predict class based on rationales
        output = enc(rationale)
        pool = pool_fn(output)
        y_hat = enc.classification_layer(pool)
        
        # compute loss and regularization term
        znorm = torch.sum(sample[:,1])
        zdist = torch.sum(torch.abs(sample[:-1,1] - sample[1:,1]))
        omega = (lamb1 * znorm) + (lamb2 * zdist)
        loss = xent(y_hat.unsqueeze(0), label)# + omega
        loss.backward()
        #print(y_hat,label,znorm)
        optimizer.step()
        
        #for p in parameters:
            #print(p.grad)
        optimizer.zero_grad()
        total_loss += loss.detach().cpu().numpy() 

    print('Epoch: {0}, Train Loss: {1:0.4f}, Number of tiles from last slide: {2:0.1f}'.format(e, total_loss,znorm))
        

In [138]:
for e in range(100):
    rationales_training_loop_GS(e, train_loader, gen, enc, pool_fn, lamb1, lamb2, xent, learning_rate, optimizer,params)

Epoch: 0, Train Loss: 24.0918, Number of tiles from last slide: 18.5
Epoch: 1, Train Loss: 20.9500, Number of tiles from last slide: 75.5
Epoch: 2, Train Loss: 20.5768, Number of tiles from last slide: 644.3
Epoch: 3, Train Loss: 19.1501, Number of tiles from last slide: 338.5
Epoch: 4, Train Loss: 21.4056, Number of tiles from last slide: 93.8
Epoch: 5, Train Loss: 20.9973, Number of tiles from last slide: 338.6
Epoch: 6, Train Loss: 20.5546, Number of tiles from last slide: 93.8
Epoch: 7, Train Loss: 19.9358, Number of tiles from last slide: 75.7
Epoch: 8, Train Loss: 18.9516, Number of tiles from last slide: 167.8
Epoch: 9, Train Loss: 20.3999, Number of tiles from last slide: 162.6
Epoch: 10, Train Loss: 19.0639, Number of tiles from last slide: 299.6
Epoch: 11, Train Loss: 18.1739, Number of tiles from last slide: 197.7
Epoch: 12, Train Loss: 18.4311, Number of tiles from last slide: 93.8
Epoch: 13, Train Loss: 17.4477, Number of tiles from last slide: 13.8
Epoch: 14, Train Loss: 